In [1]:
import geopandas as gpd
from geopandas import GeoDataFrame
import numpy as np
import pandas as pd
from shapely.geometry import Point
import rasterio
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import linregress
from pandas import DataFrame, Series
from matplotlib.pyplot import Axes
from osgeo import gdal

## Data wrangling

- read field sampling data
- remove unwanted columns from the field data
- rename columns to meaningful names
- created smaller database with just average results

set up lookup tables and helper functions

In [36]:
# superset of column names
all_cols = {
    'globalid': ['id', 'Unique Site ID'],
    'Site': ['site', 'Site No'],
    'plot': ['plot', 'Plot No'], 
    'Date': ['date', 'Date'], 
    'Name': ['observer', 'Assessor Name'], 
    'Fuel': ['veg_type', 'Vegetation Class'],
    'FireHistor': ['fire_hist', 'Time since Fire (y)'], 
    'latitude': ['lat', 'Latitude (°)'], 
    'longitude': ['lon', 'Longitude (°)'],
    'horaccmete': ['loc_accuracy', 'Location accuracy (m)'],
    'FuelDepth1': ['litter_d_1', 'Litter Depth: Sample 1 (mm)'],
    'Cover1': ['litter_state_1', 'Litter State: Sample 1 (presence/absence)'], 
    'NearSurfac': ['ns_h_1', 'Near surface fuel height: Sample 1 (m)'],
    'NearSurf_1': ['ns_state_1', 'Near surface fuel state: Sample 1 (absent/alive/dead)'],
    'Elevated1': ['elev_h_1', 'Elevated fuel height: Sample 1 (m)'],
    'ElevatedCo': ['elev_state_1', 'Elevated fuel state: Sample 1 (absent/alive/dead)'],
    'CanopyHeig': ['canopy_h_1', 'Canopy height: Sample 1 (m)'],
    'CanopyCove': ['canopy_cov_1', 'Canopy cover: Sample 1 (%)'],
    'FuelDepth2': ['litter_d_2', 'Litter Depth: Sample 2 (mm)'], 
    'Cover2': ['litter_state_2', 'Litter State: Sample 2 (presence/absence)'],
    'NearSurf_2': ['ns_h_2', 'Near surface fuel height: Sample 2 (m)'], 
    'NearSurf_3': ['ns_state_2', 'Near surface fuel state: Sample 2 (absent/alive/dead)'],
    'Elevated2': ['elev_h_2', 'Elevated fuel height: Sample 2 (m)'],
    'Elevated_1': ['elev_state_2', 'Elevated fuel state: Sample 2 (absent/alive/dead)'],
    'FuelDepth3': ['litter_d_3', 'Litter Depth: Sample 3 (mm)'],
    'Cover3': ['litter_state_3', 'Litter State: Sample 3 (presence/absence)'],
    'NearSurf_4': ['ns_h_3', 'Near surface fuel height: Sample 3 (m)'],
    'NearSurf_5': ['ns_state_3', 'Near surface fuel state: Sample 3 (absent/alive/dead)'],
    'Elevated3': ['elev_h_3', 'Elevated fuel height: Sample 3 (m)'],
    'Elevated_2': ['elev_state_3', 'Elevated fuel state: Sample 3 (absent/alive/dead)'],
    'CanopyHe_1': ['canopy_h_3', 'Canopy height: Sample 3 (m)'],
    'CanopyCo_1': ['canopy_cov_3', 'Canopy cover: Sample 3 (%)'],
    'FuelDepth4': ['litter_d_4', 'Litter Depth: Sample 4 (mm)'],
    'Cover4': ['litter_state_4', 'Litter State: Sample 4 (presence/absence)'],
    'NearSurf_6': ['ns_h_4', 'Near surface fuel height: Sample 4 (m)'],
    'NearSurf_7': ['ns_state_4', 'Near surface fuel state: Sample 4 (absent/alive/dead)'],
    'Elevated4': ['elev_h_4', 'Elevated fuel height: Sample 4 (m)'],
    'Elevated_3': ['elev_state_4', 'Elevated fuel state: Sample 4 (absent/alive/dead)'],
    'FuelDepth5': ['litter_d_5', 'Litter Depth: Sample 5 (mm)'],
    'Cover5': ['litter_state_5', 'Litter State: Sample 5 (presence/absence)'],
    'NearSurf_8': ['ns_h_5', 'Near surface fuel height: Sample 5 (m)'],
    'NearSurf_9': ['ns_state_5', 'Near surface fuel state: Sample 5 (absent/alive/dead)'],
    'Elevated5': ['elev_h_5', 'Elevated fuel height: Sample 5 (m)'],
    'Elevated_4': ['elev_state_5', 'Elevated fuel state: Sample 5 (absent/alive/dead)'],
    'CanopyHe_2': ['canopy_h_5', 'Canopy height: Sample 5 (m)'],
    'CanopyCo_2': ['canopy_cov_5', 'Canopy cover: Sample 5 (%)'],
    'FuelDepth6': ['litter_d_6', 'Litter Depth: Sample 6 (mm)'],
    'Cover6': ['litter_state_6', 'Litter State: Sample 6 (presence/absence)'],
    'NearSur_10': ['ns_h_6', 'Near surface fuel height: Sample 6 (m)'],
    'NearSur_11': ['ns_state_6', 'Near surface fuel state: Sample 6 (absent/alive/dead)'],
    'Elevated6': ['elev_h_6', 'Elevated fuel height: Sample 6 (m)'],
    'Elevated_5':['elev_state_6', 'Elevated fuel state: Sample 6 (absent/alive/dead)'],
    'FuelDepth7': ['litter_d_7', 'Litter Depth: Sample 7 (mm)'],
    'Cover7': ['litter_state_7', 'Litter State: Sample 7 (presence/absence)'],
    'NearSur_12': ['ns_h_7', 'Near surface fuel height: Sample 7 (m)'],
    'NearSur_13': ['ns_state_7', 'Near surface fuel state: Sample 7 (absent/alive/dead)'],
    'Elevated7': ['elev_h_7', 'Elevated fuel height: Sample 7 (m)'],
    'Elevated_6': ['elev_state_7', 'Elevated fuel state: Sample 7 (absent/alive/dead)'],
    'CanopyHe_3': ['canopy_h_7', 'Canopy height: Sample 7 (m)'],
    'CanopyCo_3': ['canopy_cov_7', 'Canopy cover: Sample 7 (%)'],
    'FuelDepth8': ['litter_d_8', 'Litter Depth: Sample 8 (mm)'], 
    'Cover8': ['litter_state_8', 'Litter State: Sample 8 (presence/absence)'],
    'NearSur_14': ['ns_h_8', 'Near surface fuel height: Sample 8 (m)'],
    'NearSur_15': ['ns_state_8', 'Near surface fuel state: Sample 8 (absent/alive/dead)'],
    'Elevated8': ['elev_h_8', 'Elevated fuel height: Sample 8 (m)'],
    'Elevated_7': ['elev_state_8', 'Elevated fuel state: Sample 8 (absent/alive/dead)'],
    'FuelDepth9': ['litter_d_9', 'Litter Depth: Sample 9 (mm)'],
    'Cover9': ['litter_state_9', 'Litter State: Sample 9 (presence/absence)'],
    'NearSur_16': ['ns_h_9', 'Near surface fuel height: Sample 9 (m)'],
    'NearSur_17': ['ns_state_9', 'Near surface fuel state: Sample 9 (absent/alive/dead)'],
    'Elevated9': ['elev_h_9', 'Elevated fuel height: Sample 9 (m)'],
    'Elevated_8': ['elev_state_9', 'Elevated fuel state: Sample 9 (absent/alive/dead)'],
    'CanopyHe_4': ['canopy_h_9', 'Canopy height: Sample 9 (m)'],
    'CanopyCo_4': ['canopy_cov_9', 'Canopy cover: Sample 9 (%)'],
    'FuelDept_1': ['litter_d_10', 'Litter Depth: Sample 10 (mm)'],
    'Cover10': ['litter_state_10', 'Litter State: Sample 10 (presence/absence)'],
    'NearSur_18': ['ns_h_10', 'Near surface fuel height: Sample 10 (m)'],
    'NearSur_19': ['ns_state_10', 'Near surface fuel state: Sample 10 (absent/alive/dead)'],
    'Elevated10': ['elev_h_10', 'Elevated fuel height: Sample 10 (m)'],
    'Elevated_9': ['elev_state_10', 'Elevated fuel state: Sample 10 (absent/alive/dead)'],
    'AverageFue': ['litter_d_m', 'Mean litter depth (mm)'],
    'AverageSur': ['litter_state_count', 'Litter cover P/A count (/10)'],
    'AveragePer': ['litter_cov', 'Litter cover (%)'],
    'FuelLoad': ['litter_load', 'Litter fuel load (t/ha)'],
    'Fuel_Hazar': ['s_fhr', 'Surface fuel hazard rating'], 
    'averageNSh': ['ns_h_m', 'Mean near surface height (m)'], 
    'averageNSc': ['ns_state_count', 'Near Surface P/A count (/10)'], 
    'averageN_1': ['ns_cov', 'Near surface cover (%)'], 
    'averageNSd': ['ns_dead_count', 'Near surface dead count'], 
    'percentage': ['ns_dead_%', 'Near surface dead as percentage of near surface present'], 
    'nearsur_20': ['_ns_fhr', 'Near surface Fuel Hazard Rating'], # repeat?
    'Near_Surfa': ['ns_fhr', 'Near surface Fuel Hazard Rating'], 
    'nearsur_21': ['ns_load', 'Near surface fuel load (t/ha)'], 
    'combined': ['s&ns_fhr', 'Combined surface and near surface Fuel Hazard Rating'], 
    'averageele': ['elev_h_m', 'Mean elevated height (m)'], 
    'averagee_1': ['elev_state_count', 'Elevated P/A count (/10)'], 
    'averagee_2': ['elev_cov', 'elevated cover (%)'], 
    'averagee_3': ['elev_dead_count', 'Elevated dead count'],
    'elevatedpe': ['elev_dead_%', 'elevated dead as percentage of elevated present'], 
    'elevated_f': ['_elev_fhr', 'Elevated Fuel Hazard Rating'], # repeat?
    'elevated_h': ['elev_fhr', 'Elevated Fuel Hazard Rating'], 
    'elevate_10': ['elev_load', 'Elevated fuel load (t/ha)'], 
    'bark_type': ['bark_type', 'Bark type'], 
    'bark_fuel': ['bark_haz&type', 'Bark hazard rating and type'], 
    'bark_hazar': ['bark_haz', 'Bark hazard rating'], 
    'bark_fuell': ['bark_load', 'Bark fuel load (t/ha)'], 
    'height_ave': ['canopy_h_m', 'Mean canopy height (m)'], 
    'cannopy_av':['canopy_cov_m', 'Mean canopy cover (%)'], 
    'plotgood': ['representative', 'Plot is representative of area (Y/N)'], 
    'comments': ['comments', 'Comments'],
    'geometry': ['geometry', 'Shapefile geometry'],
}

# column names and descriptions
labels = {
    'afo_cc': 'AFO Crown Cover %',
    'afo_lfd': 'AFO Ladder Fuel Density',
    'afo_ch': 'AFO Canopy Height (m)',
    'afo_cbh': 'AFO Canopy Base Height (m)',
    'afo_litter': 'AFO litter fuel load (t/ha)', 
    'afo_surface': 'AFO surface fuel load (t/ha)', 
    'afo_elevated': 'AFO elevated fuel  load (t/ha)', 
    'afo_bark': 'AFO bark fuel load (t/ha)'
}

#smaller working set of data - extraneous columns removed
subset = [
    'id', 'site', 'plot', 'observer', 'veg_type', 'fire_hist', 'loc_accuracy', 
    'litter_d_m', 'litter_cov', 'litter_load', 's_fhr', 
    'ns_h_m', 'ns_cov',#'ns_fhr', 'ns_load', 
    's&ns_fhr', 
    'elev_h_m', 'elev_cov', 'elev_load',# 'elev_fhr',
    'bark_type', 'bark_load',# 'bark_haz', 
    'canopy_h_m', 'canopy_cov_m', 
    'representative', 'comments', 'geometry'
]


In [37]:
# helper functions
def read_afo(paths: dict, coord_df: GeoDataFrame, index: int = None, offset: int = 0) -> Series:
    """samples single and multiband AFO geotiffs

    Args:
        paths (dict): region names and relative path to geotiffs
        coord_df (GeoDataFrame): gdf containing sample coordinates as `geometry`
        index (int, optional): for multiband data the index of the band to sample
        offset (int, optional): generates random offsets to test robustness of data
            sampling to location. Maximum offset in x or y is `offset`

    Returns:
        Series: sampled data
    """

    coord_df = coord_df[['geometry']]
    for loc, path in paths.items():
        with rasterio.open(path, 'r') as src:
            print(f'Input CRS is: {src.crs}')
            coord_df = coord_df.to_crs(src.crs)
            site_coords = [
                (x,y) for x,y in zip(
                    coord_df['geometry'].x - offset, coord_df['geometry'].y
                )
            ]
            coord_df[loc] = -9999
            if index is None:
                coord_df[loc] = [x for x in src.sample(site_coords)]
            else:
                coord_df[loc] = [x[index] for x in src.sample(site_coords)]
    
    return coord_df[paths.keys()].max(axis=1)

def read_rfs(path: str, coord_df: GeoDataFrame) -> Series:
    """reads RFS corporate data in geotiff form

    Args:
        path (str): path to the geotiff file
        coord_df (GeoDataFrame): gdf containing sample coordinates as `geometry`

    Returns:
        Series: sampled data
    """
    coord_df = coord_df[['geometry']]
    with rasterio.open(path, 'r') as src:
        print(f'Input CRS is: {src.crs}')
        coord_df = coord_df.to_crs(src.crs)
        site_coords = [
            (x,y) for x,y in zip(coord_df['geometry'].x, coord_df['geometry'].y)
        ]
        coord_df['sample_data'] = [x[0] for x in src.sample(site_coords)]
    return coord_df['sample_data']

def reg_plot(x: str, y: str, df: DataFrame, hue: str=None) -> Axes:
    """Regression plot with stats. Points coloured by `hue`

    Args:
        x (str): df column name for x values
        y (str): df column name for y values
        df (DataFrame):
        hue (str): df column name to use to colour the values. Defaults to `None` 
    """
    _df = df[[x, y]].dropna()
    slope, intercept, r_value, p_value, std_err = linregress(_df[x], _df[y])
    stats = f'pearson r: {r_value:.2f} \n r sq: {r_value**2:.2f}'
    grid = sns.lmplot(x=x, y=y, data=df, hue=hue, fit_reg=False, height=8, aspect=1.5)
    ax = grid.axes[0, 0]
    sns.regplot(x=x, y=y, data=df, scatter=False, ax=ax)
    ax.text(0.1, 0.9, stats, ha='center', va='center', transform=ax.transAxes)
    ax.set(xlabel=labels[x], ylabel=labels[y])
    
    return ax

def reg_plots(x: str, y: str, df: DataFrame, hue: str=None) -> sns.FacetGrid:
    """creates a series of linear regression with axis labels and calculates
    descriptive statistics.

    Args:
        x (str): df column name for x values
        y (str): df column name for y values
        df (DataFrame):
        hue (str, optional): df column name to define subsets. Defaults to `None`.
    """
    subsets = set(df[hue])
    subsets.discard(None)
    regressions = sns.FacetGrid(df, col=hue, col_order=subsets, hue=hue, height=5, aspect=1.2, col_wrap=2)
    regressions.map(sns.regplot, x, y)
    regressions.set_ylabels(labels[y])
    regressions.set_xlabels(labels[x])
    
    for ax, subset in zip(regressions.axes.flat, subsets):
        _df = df.loc[df[hue]==subset]
        _df = _df[[x, y]].dropna()
        if _df.shape[0] > 5: # make sure _df not too small
            slope, intercept, r_value, p_value, std_err = linregress(_df[x], _df[y], )
            stats = f'pearson r: {r_value:.2f} \n r sq: {r_value**2:.2f}'
            ax.text(0.15, 0.9, stats, ha='center', va='center', transform=ax.transAxes)
    
    return regressions

def rasters_resample(path_dict: dict, res: float) -> dict:
    """resamples geotiffs to resolution = `res`.

    Args:
        path_dict (dict): dictionary containing path to input geotifs.
        res (float): resolution in projected units of resultant geotiff

    Returns:
        dict: dictionary containing path to input geotifs.
    """
    out_paths = {}
    for key, path in path_dict.items():
        outpath = f'{path.split(".")[0]}_{res}.tif'
        gdal.Translate(
            outpath, path,
            options=f'-of GTiff -tr {res} {res} -r bilinear'
        )
        out_paths[key] = outpath 
    return out_paths

read the site and field sampling geodata

In [41]:
path = 'spatial_data/Field_Data/Form_2.shp'
# path = 'spatial_data/Field_Data/Fuel_Sampling___Version_0_1.shp'
gdf = gpd.read_file(path)
gdf.shape

(19, 122)

In [42]:
gdf = gdf[gdf['geometry'].x > 0] # drop sites with no location data
gdf.shape

(16, 122)

rename columns to something meaningful (associated with longer description in dictionary)

In [43]:
rename_dict = {} # used to cut down and rename columns
for key, [field, desc] in all_cols.items():
    rename_dict[key] = field
    labels[field] = desc
gdf = gdf.rename(columns=rename_dict)

the survey app deals poorly with missing data so recalculate means and percentages

In [44]:
quantitative_fields = ['canopy_cov', 'canopy_h', 'elev_h', 'litter_d', 'ns_h']
presence_absence_fields = ['elev_state', 'litter_state', 'ns_state']

for field in quantitative_fields:
    cols = [string for string in list(gdf.columns.values) if field in string]
    gdf[f'{field}_m'] = gdf[cols[:-1]].mean(axis=1)

for field in presence_absence_fields:
    cols = [string for string in list(gdf.columns.values) if field in string]
    gdf[f'{field}_count'] = (
        gdf[gdf[cols[:-1]] == 'alive'].count(axis=1) + 
        gdf[gdf[cols[:-1]] == 'dead'].count(axis=1) + 
        gdf[gdf[cols[:-1]] == 'yes'].count(axis=1)
    )
    prefix=field.split('_')[0]
    gdf[f'{prefix}_cov'] = gdf[f'{field}_count']*10

cut down the number of columns

In [45]:
gdf = gdf[subset]
# sort it just because we can :)
gdf.sort_values(by=['site', 'plot'], inplace=True)
gdf.head()

,id,site,plot,observer,veg_type,fire_hist,loc_accuracy,litter_d_m,litter_cov,litter_load,...,elev_h_m,elev_cov,elev_load,bark_type,bark_load,canopy_h_m,canopy_cov_m,representative,comments,geometry
8,5ad349fb-92af-41c4-b31e-2065470fa68c,EC30,1,David and Jon,DSF,3,101.829186,4.0,90,5,...,0.58,50,6,Stringy,0,6.25,10.0,yes,None,POINT (150.37026 -33.65092)
10,978c8ead-b872-4622-ba68-442cb781b107,EC30,3,DJ,Heath,3,4.695331,0.0,0,1.25,...,0.28,30,10,None,0,0.00,0.0,yes,None,POINT (150.36966 -33.65101)
9,3c549bc8-6be6-46eb-a9a8-d812fbdd913c,EC30,None,David and Jon,DSF,3,4.764533,7.0,100,5,...,0.70,90,6,Stringy,0,3.75,5.0,yes,None,POINT (150.37050 -33.65093)
11,8157a286-29aa-45ea-9da7-15f7cc889854,EC31,1,Dave and Jon,DSF,6,4.674024,21.0,100,10,...,0.77,90,10,Stringy,5,2.50,5.0,yes,None,POINT (150.37060 -33.66621)
12,27d9c632-a891-4b61-9c13-4d6c54881e26,EC31,2,Dave,DSF,6,4.703985,37.0,100,15,...,0.44,50,10,Stringy,5,6.75,20.0,yes,None,POINT (150.37079 -33.66660)


shapefile data are imported as objects so need to convert to data types that can be used for correlation and other analyses

In [46]:
gdf= gdf.replace('BlanK', np.nan) # some missing values

numeric = [
    'litter_d_m', 'litter_cov', 'litter_load', 
    'ns_h_m', 'ns_cov',# 'ns_load', 
    'elev_h_m', 'elev_cov', 'elev_load', 
    'bark_load', 
    'canopy_h_m', 'canopy_cov_m', 
]

for col in numeric:
    gdf[col] = pd.to_numeric(gdf[col])
gdf.dtypes

id                  object
site                object
plot                object
observer            object
veg_type            object
fire_hist           object
loc_accuracy       float64
litter_d_m         float64
litter_cov           int64
litter_load        float64
s_fhr               object
ns_h_m             float64
ns_cov               int64
s&ns_fhr            object
elev_h_m           float64
elev_cov             int64
elev_load            int64
bark_type           object
bark_load            int64
canopy_h_m         float64
canopy_cov_m       float64
representative      object
comments            object
geometry          geometry
dtype: object

In [47]:
# add region identifiers
gdf['region'] = np.where(
    gdf['site'].str.startswith('EC'), 'central',
        np.where(gdf['site'].str.startswith('P'), 'pilliga', 
            np.where(gdf['site'].str.startswith('SM'),'snowy', 'other')
    )
)
gdf.head()

,id,site,plot,observer,veg_type,fire_hist,loc_accuracy,litter_d_m,litter_cov,litter_load,...,elev_cov,elev_load,bark_type,bark_load,canopy_h_m,canopy_cov_m,representative,comments,geometry,region
8,5ad349fb-92af-41c4-b31e-2065470fa68c,EC30,1,David and Jon,DSF,3,101.829186,4.0,90,5.00,...,50,6,Stringy,0,6.25,10.0,yes,None,POINT (150.37026 -33.65092),central
10,978c8ead-b872-4622-ba68-442cb781b107,EC30,3,DJ,Heath,3,4.695331,0.0,0,1.25,...,30,10,None,0,0.00,0.0,yes,None,POINT (150.36966 -33.65101),central
9,3c549bc8-6be6-46eb-a9a8-d812fbdd913c,EC30,None,David and Jon,DSF,3,4.764533,7.0,100,5.00,...,90,6,Stringy,0,3.75,5.0,yes,None,POINT (150.37050 -33.65093),central
11,8157a286-29aa-45ea-9da7-15f7cc889854,EC31,1,Dave and Jon,DSF,6,4.674024,21.0,100,10.00,...,90,10,Stringy,5,2.50,5.0,yes,None,POINT (150.37060 -33.66621),central
12,27d9c632-a891-4b61-9c13-4d6c54881e26,EC31,2,Dave,DSF,6,4.703985,37.0,100,15.00,...,50,10,Stringy,5,6.75,20.0,yes,None,POINT (150.37079 -33.66660),central


## Sample the AFO geotiffs

It seems that all the geotiffs are all projected using the MGA zone 56 crs (EPSG:32756) regardless of what zone they are in but let's check this as we read them.

In [48]:
#crown cover
cc_paths = {
    'pilliga': 'spatial_data/Pilliga-vegetation-canopy_cover-2021.tif',
    'central': 'spatial_data/Centralcoast-vegetation-canopy_cover-2021.tif',
    'snowy': 'spatial_data/Southmnts-vegetation-canopy_cover-2021.tif',
}

gdf['afo_cc'] = read_afo(cc_paths, gdf)
gdf.head()

Input CRS is: EPSG:32756
Input CRS is: EPSG:32756
Input CRS is: EPSG:32756


,id,site,plot,observer,veg_type,fire_hist,loc_accuracy,litter_d_m,litter_cov,litter_load,...,elev_load,bark_type,bark_load,canopy_h_m,canopy_cov_m,representative,comments,geometry,region,afo_cc
8,5ad349fb-92af-41c4-b31e-2065470fa68c,EC30,1,David and Jon,DSF,3,101.829186,4.0,90,5.00,...,6,Stringy,0,6.25,10.0,yes,None,POINT (150.37026 -33.65092),central,53.731163
10,978c8ead-b872-4622-ba68-442cb781b107,EC30,3,DJ,Heath,3,4.695331,0.0,0,1.25,...,10,None,0,0.00,0.0,yes,None,POINT (150.36966 -33.65101),central,31.209862
9,3c549bc8-6be6-46eb-a9a8-d812fbdd913c,EC30,None,David and Jon,DSF,3,4.764533,7.0,100,5.00,...,6,Stringy,0,3.75,5.0,yes,None,POINT (150.37050 -33.65093),central,56.810474
11,8157a286-29aa-45ea-9da7-15f7cc889854,EC31,1,Dave and Jon,DSF,6,4.674024,21.0,100,10.00,...,10,Stringy,5,2.50,5.0,yes,None,POINT (150.37060 -33.66621),central,46.721088
12,27d9c632-a891-4b61-9c13-4d6c54881e26,EC31,2,Dave,DSF,6,4.703985,37.0,100,15.00,...,10,Stringy,5,6.75,20.0,yes,None,POINT (150.37079 -33.66660),central,62.519211


In [49]:
# ladder fuel density
lfd_paths = {
    'pilliga': 'spatial_data/Pilliga-vegetation-ladder_fuel_density-2021.tif',
    'central': 'spatial_data/Centralcoast-vegetation-ladder_fuel_density-2021.tif',
    'snowy': 'spatial_data/Southmnts-vegetation-ladder_fuel_density-2021.tif',
}
gdf['afo_lfd'] = read_afo(lfd_paths, gdf)
gdf.head()

Input CRS is: EPSG:32756
Input CRS is: EPSG:32756
Input CRS is: EPSG:32756


,id,site,plot,observer,veg_type,fire_hist,loc_accuracy,litter_d_m,litter_cov,litter_load,...,bark_type,bark_load,canopy_h_m,canopy_cov_m,representative,comments,geometry,region,afo_cc,afo_lfd
8,5ad349fb-92af-41c4-b31e-2065470fa68c,EC30,1,David and Jon,DSF,3,101.829186,4.0,90,5.00,...,Stringy,0,6.25,10.0,yes,None,POINT (150.37026 -33.65092),central,53.731163,13.559137
10,978c8ead-b872-4622-ba68-442cb781b107,EC30,3,DJ,Heath,3,4.695331,0.0,0,1.25,...,None,0,0.00,0.0,yes,None,POINT (150.36966 -33.65101),central,31.209862,15.043867
9,3c549bc8-6be6-46eb-a9a8-d812fbdd913c,EC30,None,David and Jon,DSF,3,4.764533,7.0,100,5.00,...,Stringy,0,3.75,5.0,yes,None,POINT (150.37050 -33.65093),central,56.810474,19.615562
11,8157a286-29aa-45ea-9da7-15f7cc889854,EC31,1,Dave and Jon,DSF,6,4.674024,21.0,100,10.00,...,Stringy,5,2.50,5.0,yes,None,POINT (150.37060 -33.66621),central,46.721088,8.510427
12,27d9c632-a891-4b61-9c13-4d6c54881e26,EC31,2,Dave,DSF,6,4.703985,37.0,100,15.00,...,Stringy,5,6.75,20.0,yes,None,POINT (150.37079 -33.66660),central,62.519211,5.667809


In [50]:
# canopy height
ch_paths = {
    'pilliga': 'spatial_data/Pilliga-vegetation-canopy_height-2021.tif',
    'central': 'spatial_data/Centralcoast-vegetation-canopy_height-2021.tif',
    'snowy': 'spatial_data/Southmnts-vegetation-canopy_height-2021.tif',
}

gdf['afo_ch'] = read_afo(ch_paths, gdf)
gdf.head()

Input CRS is: EPSG:32756
Input CRS is: EPSG:32756
Input CRS is: EPSG:32756


,id,site,plot,observer,veg_type,fire_hist,loc_accuracy,litter_d_m,litter_cov,litter_load,...,bark_load,canopy_h_m,canopy_cov_m,representative,comments,geometry,region,afo_cc,afo_lfd,afo_ch
8,5ad349fb-92af-41c4-b31e-2065470fa68c,EC30,1,David and Jon,DSF,3,101.829186,4.0,90,5.00,...,0,6.25,10.0,yes,None,POINT (150.37026 -33.65092),central,53.731163,13.559137,1.777588
10,978c8ead-b872-4622-ba68-442cb781b107,EC30,3,DJ,Heath,3,4.695331,0.0,0,1.25,...,0,0.00,0.0,yes,None,POINT (150.36966 -33.65101),central,31.209862,15.043867,0.607993
9,3c549bc8-6be6-46eb-a9a8-d812fbdd913c,EC30,None,David and Jon,DSF,3,4.764533,7.0,100,5.00,...,0,3.75,5.0,yes,None,POINT (150.37050 -33.65093),central,56.810474,19.615562,1.417030
11,8157a286-29aa-45ea-9da7-15f7cc889854,EC31,1,Dave and Jon,DSF,6,4.674024,21.0,100,10.00,...,5,2.50,5.0,yes,None,POINT (150.37060 -33.66621),central,46.721088,8.510427,14.431993
12,27d9c632-a891-4b61-9c13-4d6c54881e26,EC31,2,Dave,DSF,6,4.703985,37.0,100,15.00,...,5,6.75,20.0,yes,None,POINT (150.37079 -33.66660),central,62.519211,5.667809,21.276634


In [51]:
# canopy base height
cbh_paths = {
    'pilliga': 'spatial_data/Pilliga-vegetation-canopy_base_height-2021.tif',
    'central': 'spatial_data/Centralcoast-vegetation-canopy_base_height-2021.tif',
    'snowy': 'spatial_data/Southmnts-vegetation-canopy_base_height-2021.tif',
}
gdf['afo_cbh'] = read_afo(cbh_paths, gdf)
gdf.head()

Input CRS is: EPSG:32756
Input CRS is: EPSG:32756
Input CRS is: EPSG:32756


,id,site,plot,observer,veg_type,fire_hist,loc_accuracy,litter_d_m,litter_cov,litter_load,...,canopy_h_m,canopy_cov_m,representative,comments,geometry,region,afo_cc,afo_lfd,afo_ch,afo_cbh
8,5ad349fb-92af-41c4-b31e-2065470fa68c,EC30,1,David and Jon,DSF,3,101.829186,4.0,90,5.00,...,6.25,10.0,yes,None,POINT (150.37026 -33.65092),central,53.731163,13.559137,1.777588,0.973498
10,978c8ead-b872-4622-ba68-442cb781b107,EC30,3,DJ,Heath,3,4.695331,0.0,0,1.25,...,0.00,0.0,yes,None,POINT (150.36966 -33.65101),central,31.209862,15.043867,0.607993,0.500842
9,3c549bc8-6be6-46eb-a9a8-d812fbdd913c,EC30,None,David and Jon,DSF,3,4.764533,7.0,100,5.00,...,3.75,5.0,yes,None,POINT (150.37050 -33.65093),central,56.810474,19.615562,1.417030,1.103186
11,8157a286-29aa-45ea-9da7-15f7cc889854,EC31,1,Dave and Jon,DSF,6,4.674024,21.0,100,10.00,...,2.50,5.0,yes,None,POINT (150.37060 -33.66621),central,46.721088,8.510427,14.431993,3.958592
12,27d9c632-a891-4b61-9c13-4d6c54881e26,EC31,2,Dave,DSF,6,4.703985,37.0,100,15.00,...,6.75,20.0,yes,None,POINT (150.37079 -33.66660),central,62.519211,5.667809,21.276634,8.094801


In [52]:
# fuel loads
fuel_load_paths = {
    'pilliga': 'spatial_data/Pilliga-fuels-classes-2021-density.tif',
    'central': 'spatial_data/Centralcoast-fuels-classes-2021-density.tif',
    'snowy': 'spatial_data/Southmnts-fuels-classes-2021-density.tif',
}

strata = ['afo_litter', 'afo_surface', 'afo_elevated', 'afo_bark']

for i, stratum in enumerate(strata):
    gdf[stratum] = read_afo(fuel_load_paths, gdf, index=i)

gdf.head()

Input CRS is: EPSG:32756
Input CRS is: EPSG:32756
Input CRS is: EPSG:32756
Input CRS is: EPSG:32756
Input CRS is: EPSG:32756
Input CRS is: EPSG:32756
Input CRS is: EPSG:32756
Input CRS is: EPSG:32756
Input CRS is: EPSG:32756
Input CRS is: EPSG:32756
Input CRS is: EPSG:32756
Input CRS is: EPSG:32756


,id,site,plot,observer,veg_type,fire_hist,loc_accuracy,litter_d_m,litter_cov,litter_load,...,geometry,region,afo_cc,afo_lfd,afo_ch,afo_cbh,afo_litter,afo_surface,afo_elevated,afo_bark
8,5ad349fb-92af-41c4-b31e-2065470fa68c,EC30,1,David and Jon,DSF,3,101.829186,4.0,90,5.00,...,POINT (150.37026 -33.65092),central,53.731163,13.559137,1.777588,0.973498,5.284915,6.200949,2.449160,1.524674
10,978c8ead-b872-4622-ba68-442cb781b107,EC30,3,DJ,Heath,3,4.695331,0.0,0,1.25,...,POINT (150.36966 -33.65101),central,31.209862,15.043867,0.607993,0.500842,5.159966,6.054343,1.910804,1.189531
9,3c549bc8-6be6-46eb-a9a8-d812fbdd913c,EC30,None,David and Jon,DSF,3,4.764533,7.0,100,5.00,...,POINT (150.37050 -33.65093),central,56.810474,19.615562,1.417030,1.103186,5.347582,6.274478,2.536129,1.578815
11,8157a286-29aa-45ea-9da7-15f7cc889854,EC31,1,Dave and Jon,DSF,6,4.674024,21.0,100,10.00,...,POINT (150.37060 -33.66621),central,46.721088,8.510427,14.431993,3.958592,5.392866,6.327575,2.942374,1.833350
12,27d9c632-a891-4b61-9c13-4d6c54881e26,EC31,2,Dave,DSF,6,4.703985,37.0,100,15.00,...,POINT (150.37079 -33.66660),central,62.519211,5.667809,21.276634,8.094801,5.441453,6.384583,3.331478,2.075794


trim the dataframe down again

In [54]:
subset.extend(['afo_cc', 'afo_lfd', 'afo_ch', 'afo_cbh', 'afo_litter', 'afo_surface', 'afo_elevated', 'afo_bark'])
# gdf = gdf[[
#     'id', 'site', 'plot', 'observer', 'region', 'geometry', 'veg_type', 'fire_hist', 'loc_accuracy',
#     'litter_d_m', 'litter_cov', 'litter_load', 's_fhr', 
#     'ns_h_m', 'ns_cov', 'ns_fhr', 'ns_fhr', 'ns_load', 's&ns_fhr', 
#     'elev_h_m', 'elev_cov', 'elev_fhr', 'elev_fhr', 'elev_load', 
#     'bark_type', 'bark_haz', 'bark_load', 
#     'canopy_h_m', 'canopy_cov_m', 
#     'representative', 
#     'afo_cc', 'afo_lfd', 'afo_ch', 'afo_cbh', 'afo_litter', 'afo_surface', 'afo_elevated', 'afo_bark'
# ]]
gdf = gdf[subset]
# 'bark_haz', 
gdf

,id,site,plot,observer,veg_type,fire_hist,loc_accuracy,litter_d_m,litter_cov,litter_load,...,comments,geometry,afo_cc,afo_lfd,afo_ch,afo_cbh,afo_litter,afo_surface,afo_elevated,afo_bark
8,5ad349fb-92af-41c4-b31e-2065470fa68c,EC30,1,David and Jon,DSF,3,101.829186,4.0,90,5.00,...,None,POINT (150.37026 -33.65092),53.731163,13.559137,1.777588,0.973498,5.284915,6.200949,2.449160,1.524674
10,978c8ead-b872-4622-ba68-442cb781b107,EC30,3,DJ,Heath,3,4.695331,0.0,0,1.25,...,None,POINT (150.36966 -33.65101),31.209862,15.043867,0.607993,0.500842,5.159966,6.054343,1.910804,1.189531
9,3c549bc8-6be6-46eb-a9a8-d812fbdd913c,EC30,None,David and Jon,DSF,3,4.764533,7.0,100,5.00,...,None,POINT (150.37050 -33.65093),56.810474,19.615562,1.417030,1.103186,5.347582,6.274478,2.536129,1.578815
11,8157a286-29aa-45ea-9da7-15f7cc889854,EC31,1,Dave and Jon,DSF,6,4.674024,21.0,100,10.00,...,None,POINT (150.37060 -33.66621),46.721088,8.510427,14.431993,3.958592,5.392866,6.327575,2.942374,1.833350
12,27d9c632-a891-4b61-9c13-4d6c54881e26,EC31,2,Dave,DSF,6,4.703985,37.0,100,15.00,...,None,POINT (150.37079 -33.66660),62.519211,5.667809,21.276634,8.094801,5.441453,6.384583,3.331478,2.075794
13,e16e5cf6-969f-456c-9fea-de01f46bba45,EC31,3,Dave,DSF,3,4.647908,16.0,100,5.00,...,None,POINT (150.37198 -33.66668),58.438370,5.406912,14.291039,5.799373,5.401111,6.337249,3.132676,1.951924
14,2d96880f-7e34-4e96-aaa6-87a71bde9977,EC32,1,David F,DSF,8+,12.458665,36.0,90,15.00,...,None,POINT (150.34676 -33.70032),67.533112,7.864598,16.741819,3.479784,14.745332,16.805912,7.320443,3.687985
16,33576753-fa48-4bff-a480-b48a9b8de485,EC32,3,David and Jon,Heath,8+,5.323871,17.0,90,5.00,...,None,POINT (150.34382 -33.70101),67.098259,4.304018,9.527924,2.793477,10.079771,10.660125,6.548813,1.006274
17,7bf246bb-abf8-488c-b12d-9574a8a84571,EC33,1,Dave and Jon,Heath,8+,4.676082,10.0,90,5.00,...,Heath area surrounded by DSF,POINT (150.44504 -33.70460),55.968025,14.818193,5.753607,2.587954,14.749335,16.808718,4.939336,2.490224
15,76d012e9-c470-4651-be9b-b6a45546e914,EC33,2,David and Jon,DSF,8+,4.701087,31.0,100,15.00,...,None,POINT (150.34886 -33.69906),52.625076,9.680552,14.014324,3.556634,14.865906,16.943504,7.001962,3.527290


In [55]:
df_corr = gdf.corr(method='pearson')
df_corr

,loc_accuracy,litter_d_m,litter_cov,litter_load,ns_h_m,ns_cov,elev_h_m,elev_cov,elev_load,bark_load,canopy_h_m,canopy_cov_m,afo_cc,afo_lfd,afo_ch,afo_cbh,afo_litter,afo_surface,afo_elevated,afo_bark
loc_accuracy,1.000000,-0.370070,0.022847,-0.213093,-0.315603,0.366825,-0.128083,-0.211372,-0.029669,-0.157767,0.037205,-0.078480,-0.057921,0.168888,-0.338105,-0.320320,-0.282966,-0.277594,-0.244810,-0.183708
litter_d_m,-0.370070,1.000000,0.453967,0.911145,0.291481,-0.279345,0.303392,0.267954,0.106313,0.607119,0.268753,0.410103,0.429649,-0.416765,0.838387,0.611877,0.402710,0.402649,0.499569,0.583022
litter_cov,0.022847,0.453967,1.000000,0.396612,0.199732,-0.032184,0.277845,0.400939,-0.313568,0.228630,0.301574,0.334508,0.318673,-0.269218,0.412571,0.453723,0.258531,0.253537,0.268356,0.301721
litter_load,-0.213093,0.911145,0.396612,1.000000,0.073497,-0.244783,0.164768,0.219171,0.179354,0.600518,0.284446,0.427600,0.325515,-0.258702,0.685625,0.458240,0.308614,0.317206,0.426682,0.603349
ns_h_m,-0.315603,0.291481,0.199732,0.073497,1.000000,-0.231999,0.752298,0.624058,0.203202,-0.021852,-0.164332,-0.131156,0.432469,-0.252721,0.245154,0.062745,0.656796,0.631540,0.672792,0.287962
ns_cov,0.366825,-0.279345,-0.032184,-0.244783,-0.231999,1.000000,-0.221988,-0.098421,-0.431187,-0.037743,0.278495,0.068731,-0.007695,-0.574706,-0.139243,0.173550,-0.055849,-0.046796,0.181511,0.086707
elev_h_m,-0.128083,0.303392,0.277845,0.164768,0.752298,-0.221988,1.000000,0.791947,0.234479,-0.087952,0.030010,0.039106,0.201088,-0.020029,0.068987,-0.084212,0.538965,0.501606,0.510078,0.099616
elev_cov,-0.211372,0.267954,0.400939,0.219171,0.624058,-0.098421,0.791947,1.000000,0.183394,0.012783,0.121199,0.015519,0.377202,-0.066231,0.054894,-0.018117,0.547437,0.532917,0.559722,0.303378
elev_load,-0.029669,0.106313,-0.313568,0.179354,0.203202,-0.431187,0.234479,0.183394,1.000000,0.457568,-0.694152,-0.623104,-0.065072,0.348995,0.202853,-0.416760,-0.204391,-0.212410,0.002995,-0.095181
bark_load,-0.157767,0.607119,0.228630,0.600518,-0.021852,-0.037743,-0.087952,0.012783,0.457568,1.000000,-0.347537,-0.229929,0.075883,-0.256607,0.813803,0.288140,-0.069971,-0.055201,0.208940,0.451873


just grab the rows and columns we are interested in

In [57]:
afo_keys = [
    'afo_cc', 'afo_lfd', 'afo_ch', 'afo_cbh', 'afo_litter', 
    'afo_surface', 'afo_elevated', 'afo_bark',     
]

field_keys = [
    'litter_d_m', 'litter_cov', 'litter_load', 'ns_h_m', 'ns_cov', #'ns_load', 
    'elev_h_m', 'elev_cov', 'elev_load', 'bark_load', 
    'canopy_h_m', 'canopy_cov_m',
]

df_corr.loc[field_keys, afo_keys]

,afo_cc,afo_lfd,afo_ch,afo_cbh,afo_litter,afo_surface,afo_elevated,afo_bark
litter_d_m,0.429649,-0.416765,0.838387,0.611877,0.402710,0.402649,0.499569,0.583022
litter_cov,0.318673,-0.269218,0.412571,0.453723,0.258531,0.253537,0.268356,0.301721
litter_load,0.325515,-0.258702,0.685625,0.458240,0.308614,0.317206,0.426682,0.603349
ns_h_m,0.432469,-0.252721,0.245154,0.062745,0.656796,0.631540,0.672792,0.287962
ns_cov,-0.007695,-0.574706,-0.139243,0.173550,-0.055849,-0.046796,0.181511,0.086707
elev_h_m,0.201088,-0.020029,0.068987,-0.084212,0.538965,0.501606,0.510078,0.099616
elev_cov,0.377202,-0.066231,0.054894,-0.018117,0.547437,0.532917,0.559722,0.303378
elev_load,-0.065072,0.348995,0.202853,-0.416760,-0.204391,-0.212410,0.002995,-0.095181
bark_load,0.075883,-0.256607,0.813803,0.288140,-0.069971,-0.055201,0.208940,0.451873
canopy_h_m,0.273735,-0.380552,-0.046097,0.584269,0.330673,0.337507,0.174419,0.227599


Graph variables that should correlate

note ladder fuel density does not seem well correlated with anything :(

In [ ]:
x = 'afo_cc'
y = 'canopy_cov_m'
df = gdf
reg_plot(x, y, df, hue='region')
plt.show()
reg_plots(x, y, df, hue='region')
plt.show()
reg_plots(x, y, df, hue='veg_type')
plt.show()

In [ ]:
x = 'afo_ch'
y = 'canopy_h_m'
df = gdf
reg_plot(x, y, df, hue='region')
plt.show()
reg_plots(x, y, df, hue='region')
plt.show()
reg_plots(x, y, df, hue='veg_type')
plt.show()

In [ ]:
x = 'afo_cbh'
y = 'canopy_h_m'
df = gdf
reg_plot(x, y, df, hue='region')
plt.show()
reg_plots(x, y, df, hue='region')
plt.show()
reg_plots(x, y, df, hue='veg_type')
plt.show()

In [ ]:
x = 'afo_litter'
y = 'litter_d_m'
df = gdf
reg_plot(x, y, df, hue='region')
plt.show()
reg_plots(x, y, df, hue='region')
plt.show()
reg_plots(x, y, df, hue='veg_type')
plt.show()

In [ ]:
x = 'afo_litter'
y = 'litter_load'
df = gdf
reg_plot(x, y, df, hue='region')
plt.show()
reg_plots(x, y, df, hue='region')
plt.show()
reg_plots(x, y, df, hue='veg_type')
plt.show()

In [ ]:
x = 'afo_litter'
y = 'afo_surface'
df = gdf
reg_plot(x, y, df, hue='region')
plt.show()
reg_plots(x, y, df, hue='region')
plt.show()
reg_plots(x, y, df, hue='veg_type')
plt.show()

In [ ]:
x = 'afo_surface'
y = 'litter_d_m'
df = gdf
reg_plot(x, y, df, hue='region')
plt.show()
reg_plots(x, y, df, hue='region')
plt.show()
reg_plots(x, y, df, hue='veg_type')
plt.show()

In [ ]:
x = 'afo_elevated'
y = 'elev_cov'
df = gdf
reg_plot(x, y, df, hue='region')
plt.show()
reg_plots(x, y, df, hue='region')
plt.show()
reg_plots(x, y, df, hue='veg_type')
plt.show()

In [ ]:
x = 'afo_bark'
y = 'canopy_cov_m'
df = gdf
reg_plot(x, y, df, hue='region')
plt.show()
reg_plots(x, y, df, hue='region')
plt.show()
reg_plots(x, y, df, hue='veg_type')
plt.show()

poor correlations may be due to the fine and variable nature of the AFO data. Test how robust the results are to location by sampling with an offset of 20 m

In [ ]:
gdf['afo_cc_off'] = read_afo(cc_paths, gdf, offset=20)
labels['afo_cc_off'] = 'AFO Canopy Cover offset 20 m'
gdf.head()

In [ ]:
x = 'afo_cc'
y = 'afo_cc_off'
df = gdf
reg_plot(x, y, df, hue='region')
plt.show()
reg_plots(x, y, df, hue='region')
plt.show()
reg_plots(x, y, df, hue='veg_type')
plt.show()

If the data are so sensitive lets resample it - after a bit of playing it seems that for most of the AFO datasets the greatest improvement in r comes if we resample to a grid cell size of 30 m

In [ ]:
res = 30
lr_cc_paths = rasters_resample(cc_paths, res)
gdf['afo_cc_lr'] = read_afo(lr_cc_paths, gdf)
labels['afo_cc_lr'] = f'AFO Canopy Cover (%) {res} m resolution'
gdf.head()

In [ ]:
x = 'afo_cc_lr'
y = 'canopy_cov_m'
df = gdf
reg_plot(x, y, df, hue='region')
plt.show()
reg_plots(x, y, df, hue='region')
plt.show()
reg_plots(x, y, df, hue='veg_type')
plt.show()

In [ ]:
label = 'afo_surface_lr'
paths = rasters_resample(fuel_load_paths, res)
gdf[label] = read_afo(paths, gdf, index=1)
labels[label] = f'AFO Surface Cover {res} m resolution'
x = label
y = 'litter_d_m'
df = gdf
reg_plot(x, y, df, hue='region')
plt.show()
reg_plots(x, y, df, hue='region')
plt.show()
reg_plots(x, y, df, hue='veg_type')
plt.show()

## RFS Corporate Data

In [18]:
# def read_rfs(path: str, coord_df: GeoDataFrame) -> Series:

#     coord_df = coord_df[['geometry']]
#     with rasterio.open(path, 'r') as src:
#         print(f'Input CRS is: {src.crs}')
#         coord_df = coord_df.to_crs(src.crs)
#         site_coords = [
#             (x,y) for x,y in zip(coord_df['geometry'].x, coord_df['geometry'].y)
#         ]
#         coord_df['sample_data'] = [x[0] for x in src.sample(site_coords)]
#     return coord_df['sample_data']

In [58]:
path = 'spatial_data/RFS/Bark_FuelTypeV211_202109201.tif'
gdf['rfs_bark'] = read_rfs(path,gdf)
labels['rfs_bark'] = 'RFS Bark Fuel Load 02109201'
path = 'spatial_data/RFS/Canopy_FuelTypeV2111.tif'
gdf['rfs_canopy'] = read_rfs(path,gdf)
labels['rfs_canopy'] = 'RFS Canopy Fuel Load 02109201'
path = 'spatial_data/RFS/Elevated_FuelTypeV211_202109201.tif'
gdf['rfs_elevated'] = read_rfs(path,gdf)
labels['rfs_elevated'] = 'RFS Elevated Fuel Load 02109201'
path = 'spatial_data/RFS/Surface_FuelTypeV211_202109201.tif'
gdf['rfs_surface'] = read_rfs(path,gdf)
labels['rfs_surface'] = 'RFS Surface Fuel Load 02109201'
gdf.head()

Input CRS is: EPSG:3308
Input CRS is: EPSG:3308
Input CRS is: EPSG:3308
Input CRS is: EPSG:3308


,id,site,plot,observer,veg_type,fire_hist,loc_accuracy,litter_d_m,litter_cov,litter_load,...,afo_ch,afo_cbh,afo_litter,afo_surface,afo_elevated,afo_bark,rfs_bark,rfs_canopy,rfs_elevated,rfs_surface
8,5ad349fb-92af-41c4-b31e-2065470fa68c,EC30,1,David and Jon,DSF,3,101.829186,4.0,90,5.00,...,1.777588,0.973498,5.284915,6.200949,2.449160,1.524674,1.302720,3.5,2.084656,5.936976
10,978c8ead-b872-4622-ba68-442cb781b107,EC30,3,DJ,Heath,3,4.695331,0.0,0,1.25,...,0.607993,0.500842,5.159966,6.054343,1.910804,1.189531,0.000000,0.0,2.421469,2.424473
9,3c549bc8-6be6-46eb-a9a8-d812fbdd913c,EC30,None,David and Jon,DSF,3,4.764533,7.0,100,5.00,...,1.417030,1.103186,5.347582,6.274478,2.536129,1.578815,1.302720,3.5,2.084656,5.936976
11,8157a286-29aa-45ea-9da7-15f7cc889854,EC31,1,Dave and Jon,DSF,6,4.674024,21.0,100,10.00,...,14.431993,3.958592,5.392866,6.327575,2.942374,1.833350,1.657094,3.5,3.354906,10.117001
12,27d9c632-a891-4b61-9c13-4d6c54881e26,EC31,2,Dave,DSF,6,4.703985,37.0,100,15.00,...,21.276634,8.094801,5.441453,6.384583,3.331478,2.075794,1.657094,3.5,3.354906,10.117001


In [59]:
df_corr = gdf.corr(method='pearson')
df_corr

,loc_accuracy,litter_d_m,litter_cov,litter_load,ns_h_m,ns_cov,elev_h_m,elev_cov,elev_load,bark_load,...,afo_ch,afo_cbh,afo_litter,afo_surface,afo_elevated,afo_bark,rfs_bark,rfs_canopy,rfs_elevated,rfs_surface
loc_accuracy,1.000000,-0.370070,0.022847,-0.213093,-0.315603,0.366825,-0.128083,-0.211372,-0.029669,-0.157767,...,-0.338105,-0.320320,-0.282966,-0.277594,-0.244810,-0.183708,0.014989,0.148067,-0.300599,-0.200014
litter_d_m,-0.370070,1.000000,0.453967,0.911145,0.291481,-0.279345,0.303392,0.267954,0.106313,0.607119,...,0.838387,0.611877,0.402710,0.402649,0.499569,0.583022,0.664036,0.446983,0.314861,0.842939
litter_cov,0.022847,0.453967,1.000000,0.396612,0.199732,-0.032184,0.277845,0.400939,-0.313568,0.228630,...,0.412571,0.453723,0.258531,0.253537,0.268356,0.301721,0.404903,0.529129,0.098105,0.408458
litter_load,-0.213093,0.911145,0.396612,1.000000,0.073497,-0.244783,0.164768,0.219171,0.179354,0.600518,...,0.685625,0.458240,0.308614,0.317206,0.426682,0.603349,0.807591,0.571481,0.186951,0.897530
ns_h_m,-0.315603,0.291481,0.199732,0.073497,1.000000,-0.231999,0.752298,0.624058,0.203202,-0.021852,...,0.245154,0.062745,0.656796,0.631540,0.672792,0.287962,-0.276789,-0.480885,0.820359,0.132184
ns_cov,0.366825,-0.279345,-0.032184,-0.244783,-0.231999,1.000000,-0.221988,-0.098421,-0.431187,-0.037743,...,-0.139243,0.173550,-0.055849,-0.046796,0.181511,0.086707,0.051778,0.061621,-0.132966,-0.141743
elev_h_m,-0.128083,0.303392,0.277845,0.164768,0.752298,-0.221988,1.000000,0.791947,0.234479,-0.087952,...,0.068987,-0.084212,0.538965,0.501606,0.510078,0.099616,-0.031060,-0.114683,0.505194,0.329158
elev_cov,-0.211372,0.267954,0.400939,0.219171,0.624058,-0.098421,0.791947,1.000000,0.183394,0.012783,...,0.054894,-0.018117,0.547437,0.532917,0.559722,0.303378,0.095806,-0.064769,0.582324,0.365905
elev_load,-0.029669,0.106313,-0.313568,0.179354,0.203202,-0.431187,0.234479,0.183394,1.000000,0.457568,...,0.202853,-0.416760,-0.204391,-0.212410,0.002995,-0.095181,-0.173022,-0.386409,0.323290,-0.035138
bark_load,-0.157767,0.607119,0.228630,0.600518,-0.021852,-0.037743,-0.087952,0.012783,0.457568,1.000000,...,0.813803,0.288140,-0.069971,-0.055201,0.208940,0.451873,0.423770,0.234579,0.147741,0.411199


In [60]:
rfs_keys = ['rfs_bark', 'rfs_canopy', 'rfs_elevated', 'rfs_surface']

In [61]:
df_corr.loc[field_keys, afo_keys]

,afo_cc,afo_lfd,afo_ch,afo_cbh,afo_litter,afo_surface,afo_elevated,afo_bark
litter_d_m,0.429649,-0.416765,0.838387,0.611877,0.402710,0.402649,0.499569,0.583022
litter_cov,0.318673,-0.269218,0.412571,0.453723,0.258531,0.253537,0.268356,0.301721
litter_load,0.325515,-0.258702,0.685625,0.458240,0.308614,0.317206,0.426682,0.603349
ns_h_m,0.432469,-0.252721,0.245154,0.062745,0.656796,0.631540,0.672792,0.287962
ns_cov,-0.007695,-0.574706,-0.139243,0.173550,-0.055849,-0.046796,0.181511,0.086707
elev_h_m,0.201088,-0.020029,0.068987,-0.084212,0.538965,0.501606,0.510078,0.099616
elev_cov,0.377202,-0.066231,0.054894,-0.018117,0.547437,0.532917,0.559722,0.303378
elev_load,-0.065072,0.348995,0.202853,-0.416760,-0.204391,-0.212410,0.002995,-0.095181
bark_load,0.075883,-0.256607,0.813803,0.288140,-0.069971,-0.055201,0.208940,0.451873
canopy_h_m,0.273735,-0.380552,-0.046097,0.584269,0.330673,0.337507,0.174419,0.227599


In [62]:
df_corr.loc[field_keys, rfs_keys]

,rfs_bark,rfs_canopy,rfs_elevated,rfs_surface
litter_d_m,0.664036,0.446983,0.314861,0.842939
litter_cov,0.404903,0.529129,0.098105,0.408458
litter_load,0.807591,0.571481,0.186951,0.897530
ns_h_m,-0.276789,-0.480885,0.820359,0.132184
ns_cov,0.051778,0.061621,-0.132966,-0.141743
elev_h_m,-0.031060,-0.114683,0.505194,0.329158
elev_cov,0.095806,-0.064769,0.582324,0.365905
elev_load,-0.173022,-0.386409,0.323290,-0.035138
bark_load,0.423770,0.234579,0.147741,0.411199
canopy_h_m,0.489497,0.538820,-0.155485,0.460912


In [63]:
df_corr.loc[afo_keys, rfs_keys]

,rfs_bark,rfs_canopy,rfs_elevated,rfs_surface
afo_cc,0.302038,-0.011197,0.565413,0.365143
afo_lfd,-0.220801,-0.032188,-0.387787,-0.283512
afo_ch,0.433248,0.256290,0.292033,0.518648
afo_cbh,0.428409,0.389491,0.086915,0.437369
afo_litter,0.162883,-0.086111,0.667951,0.531396
afo_surface,0.182989,-0.074259,0.661019,0.539154
afo_elevated,0.291190,-0.111040,0.787505,0.562361
afo_bark,0.585830,0.237261,0.465337,0.711654


In [ ]:
x = 'afo_bark'
y = 'rfs_bark'
reg_plot(x, y, gdf, hue='region')
plt.show()
reg_plots(x, y, gdf, hue='region')
plt.show()
reg_plots(x, y, gdf, hue='veg_type')
plt.show()

In [ ]:
x = 'afo_elevated'
y = 'rfs_elevated'
reg_plot(x, y, gdf, hue='region')
plt.show()
reg_plots(x, y, gdf, hue='region')
plt.show()
reg_plots(x, y, gdf, hue='veg_type')
plt.show()

In [ ]:
x = 'afo_surface'
y = 'rfs_surface'
reg_plot(x, y, gdf, hue='region')
plt.show()
reg_plots(x, y, gdf, hue='region')
plt.show()

In [ ]:
x = 'afo_surface'
y = 'rfs_surface'
reg_plot(x, y, gdf, hue='region')
plt.show()
reg_plots(x, y, gdf, hue='region')
plt.show()
reg_plots(x, y, gdf, hue='veg_type')
plt.show()

In [64]:
gdf.to_csv('AFO_truthiness_2.csv')